## Is there any weighting going on in the MN poll? Let's see.

In [1]:
import pandas as pd

In [2]:
questions = pd.read_excel('data/MN1018Questions.xlsx')
questions.head()

,question_num,bad_question_num,question_text,a_1,a_2,a_3,a_4,a_5,a_6
0,Q1,Q4,"Do you have a favorable, unfavorable or neutra...",Recognize favorable,Recognize unfavorable,Recognize neutral,Don't recognize,NaN,NaN
1,Q10,Q10,If the 2018 general election for Minnesota gov...,"Tim Walz, the Democratic-Farmer-Labor candidate","Jeff Johnson, the Republican candidate","Chris Wright, the Grassroots-Legalize Cannabis...","Josh Welter, the Libertarian candidate",Undecided,NaN
2,Q11,Q11,If the 2018 general election for Minnesota att...,"Keith Ellison, the Democratic-Farmer-Labor can...","Doug Wardlow, the Republican candidate","Noah Johnson, the Grassroots – Legalize Cannab...",Undecided,NaN,NaN
3,Q12,Q12,If the 2018 general election for U.S. senator ...,"Amy Klobuchar, the Democratic-Farmer-Labor can...","Jim Newberger, the Republican candidate","Dennis Schuller, the Legal Marijuana Now candi...","Paula Overby, the Green Party candidate",Undecided,NaN
4,Q13,Q13,If the 2018 special election for U.S. senator ...,"Tina Smith, the Democratic-Farmer-Labor candidate","Karin Housley, the Republican candidate","Sarah Wellington, the Legal Marijuana Now cand...","Jerry Trooien, the unaffiliated candidate",Undecided,NaN


In [3]:
# Each row is a response from a single respondant.
raw_responses = pd.read_excel('data/MN1018Data.xlsx')
raw_responses.insert(0, 'respondent_id', range(0, 0 + len(raw_responses)))
raw_responses

,respondent_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q32A
0,0,1,3,2,1,2,1,1,2,1,...,1,1,1,2,1,4,2,2,1,1.0
1,1,1,4,2,1,2,1,1,3,1,...,1,1,1,1,1,4,3,1,3,2.0
2,2,1,4,4,3,2,1,4,4,1,...,1,3,3,2,1,2,1,2,1,1.0
3,3,3,1,4,4,4,3,4,3,3,...,2,1,1,4,2,1,5,1,2,2.0
4,4,1,3,4,1,2,1,1,2,1,...,1,1,1,3,1,4,1,2,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,2,4,4,4,1,2,2,4,2,...,1,1,1,1,2,4,3,2,2,2.0
796,796,1,4,4,1,2,1,3,4,1,...,1,1,1,2,1,2,5,2,3,2.0
797,797,1,3,2,1,2,1,3,4,1,...,1,1,2,2,1,2,3,1,1,2.0
798,798,1,4,2,1,2,1,1,4,1,...,1,1,1,2,1,4,4,1,3,2.0


In [4]:
response_totals = raw_responses.count()
response_totals.head()

respondent_id    800
Q1               800
Q2               800
Q3               800
Q4               800
dtype: int64

In [5]:
def display_question_results(q=1):
    '''Output a data frame of percentages for each question'''
    which_q = 'Q{0}'.format(q)
#     print(which_q)
    
    q_info = questions[questions['question_num'] == which_q]
    q_title = list(q_info['question_text'])[0]
    
    # Group by response num
    q_totals = raw_responses[['respondent_id', which_q]].groupby([which_q]).agg('count').reset_index()
    q_totals.rename(columns={'respondent_id': 'count'}, inplace=True)
    
    # Calculate percentage
    q_totals['pct'] = q_totals['count'] / response_totals[which_q]
    
    # Match to response texts. Could do this en masse beforehand, but at this scale meh no real performance issue
    a_rows = q_info.drop(
        ['question_num', 'bad_question_num', 'question_text'],
        axis=1
    ).transpose().dropna().reset_index()
    
    a_rows.rename(columns={a_rows.columns[1]: 'response_text'}, inplace=True)

    a_rows['response_num'] = a_rows['index'].str.replace('a_', '').astype(int)
    q_totals = q_totals.merge(
        a_rows, 
        how="left", 
        left_on=which_q, 
        right_on='response_num'
    ).drop(columns=['index', which_q])
    
    print('{0}: {1}'.format(which_q, q_title))
#     print(q_totals.to_string())
    print(q_totals[[
        'response_text',
        'pct',
        'count',
        'response_num'
    ]].to_string())
    print('\n')

In [6]:
for q in range(1, 35):
    display_question_results(q)

Q1: Do you have a favorable, unfavorable or neutral opinion of Amy Klobuchar?
           response_text      pct  count  response_num
0    Recognize favorable  0.51500    412             1
1  Recognize unfavorable  0.25625    205             2
2      Recognize neutral  0.22125    177             3
3        Don't recognize  0.00750      6             4


Q2: Do you have a favorable, unfavorable or neutral opinion of Jim Newberger?
           response_text      pct  count  response_num
0    Recognize favorable  0.11000     88             1
1  Recognize unfavorable  0.04250     34             2
2      Recognize neutral  0.24625    197             3
3        Don't recognize  0.60125    481             4


Q3: Do you have a favorable, unfavorable or neutral opinion of Karin Housley?
           response_text      pct  count  response_num
0    Recognize favorable  0.26875    215             1
1  Recognize unfavorable  0.13875    111             2
2      Recognize neutral  0.23875    191       

Q34: REGION
        response_text      pct  count  response_num
0     Hennepin/Ramsey  0.31875    255             1
1       Metro Suburbs  0.28750    230             2
2  Southern Minnesota  0.20000    160             3
3  Northern Minnesota  0.19375    155             4


